<a href="https://colab.research.google.com/github/AftabUdaipurwala/MachineLearningBasics/blob/ClassificationModels/Hyperparameter_Tuning_Grid_Search_IRIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import warnings
warnings.filterwarnings("ignore")

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
!ls '/content/drive/MyDrive/py-master/ML/15_gridsearch/'

15_grid_search.ipynb  Exercise	exercise.md  iris_petal_sepal.png


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


In [33]:
df =load_iris()
df1 = pd.DataFrame(df.data)
df1.columns = df.feature_names
df1['Class'] = df.target
df1['Flower'] = pd.Categorical.from_codes(df.target,df.target_names)
df1.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Class,Flower
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [34]:
df1.columns =['SL','SW','PL','PW','Target','Species']
df1.head()
x= df1.drop(['Target','Species'],axis=1)
y=df1.Target

In [35]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=123)


In [36]:
model=LogisticRegression()

In [40]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier

lis =[SVC(),LogisticRegression(),RandomForestClassifier(),MultinomialNB(),GaussianNB(),DecisionTreeClassifier()]
for i in lis:
  model =i
  model.fit(xtrain,ytrain)
  score = model.score(xtest,ytest)
  model_name = type(model).__name__
  print("Model ", model_name,"has Score of ",score)


Model  SVC has Score of  0.9666666666666667
Model  LogisticRegression has Score of  1.0
Model  RandomForestClassifier has Score of  0.9333333333333333
Model  MultinomialNB has Score of  0.7666666666666667
Model  GaussianNB has Score of  0.9666666666666667
Model  DecisionTreeClassifier has Score of  0.9666666666666667


In [43]:
from sklearn.model_selection import cross_val_score


In [53]:
cross_val_score(SVC(kernel='linear',C=10, gamma='auto'), x,y, cv=5).mean().round(2)

0.97

In [52]:
cross_val_score(SVC(kernel='rbf',C=10, gamma='auto'), x,y, cv=5).mean().round(2)

0.98

In [54]:
# doing it in for loop
lis =['linear','rbf','sigmoid','polynomial']
cost=[1,10,20,30]
for i in lis:
  for j in cost:
    cvscore=cross_val_score(SVC(kernel=i,C=j, gamma='auto'), x,y, cv=5).mean().round(2)
    print("Model with SVM Kernel",i,"cost as",j,"score is ",cvscore)

Model with SVM Kernel linear cost as 1 score is  0.98
Model with SVM Kernel linear cost as 10 score is  0.97
Model with SVM Kernel linear cost as 20 score is  0.97
Model with SVM Kernel linear cost as 30 score is  0.96
Model with SVM Kernel rbf cost as 1 score is  0.98
Model with SVM Kernel rbf cost as 10 score is  0.98
Model with SVM Kernel rbf cost as 20 score is  0.97
Model with SVM Kernel rbf cost as 30 score is  0.96
Model with SVM Kernel sigmoid cost as 1 score is  0.09
Model with SVM Kernel sigmoid cost as 10 score is  0.09
Model with SVM Kernel sigmoid cost as 20 score is  0.09
Model with SVM Kernel sigmoid cost as 30 score is  0.09
Model with SVM Kernel polynomial cost as 1 score is  nan
Model with SVM Kernel polynomial cost as 10 score is  nan
Model with SVM Kernel polynomial cost as 20 score is  nan
Model with SVM Kernel polynomial cost as 30 score is  nan


In [67]:
# doing it using GridsearchCV
from sklearn.model_selection import GridSearchCV
clf= GridSearchCV(SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear'],
   
} , cv=5,
    return_train_score=False
)

clf.fit(df.data,df.target)
clfdf= pd.DataFrame(clf.cv_results_)
clfdf.sort_values('mean_test_score', ascending=False)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000700,0.000100,0.000395,0.000048,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000457,0.000017,0.000280,0.000005,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000566,0.000015,0.000302,0.000009,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000480,0.000036,0.000299,0.000022,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000688,0.000033,0.000400,0.000041,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000520,0.000018,0.000319,0.000007,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [70]:
clfdf[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [ ]:
dir(clf)

In [72]:
clf.best_score_

0.9800000000000001

In [73]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [78]:
#Randomized Search CV
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(
    SVC(gamma='auto'),
    {
     'C':[1,10,20],
     'kernel':['linear','rbf']
    },
    cv=5,
    return_train_score=False,
    n_iter=2
)
rs.fit(xtrain,ytrain)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.983333
1,10,linear,0.966667


In [80]:
# now finding out the best model
model_params = {
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
            'C': [1,10,20],
            'kernel':['rbf','linear']
        }
    },
    'RandomForest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10]
        }
    },
        'LogisticRegresson':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,5,10]
        }
    }
}

In [84]:
scores =[]
for model_name,mp in model_params.items():
  clf= GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
  clf.fit(df.data,df.target)
  scores.append({
      'model':model_name,
      'best_score':clf.best_score_,
      'best_params':clf.best_params_
  })
pd.DataFrame(scores)

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,RandomForest,0.960000,{'n_estimators': 10}
2,LogisticRegresson,0.966667,{'C': 5}
